In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 4.7 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://unriffled-haematoid-rochel.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://unriffled-haematoid-rochel.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [8]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學 (Ming Hsin University of Science and Technology, 簡稱明新科大) 是一所位於**台灣新竹縣**的私立科技大學。

**主要特色與簡介：**

1.  **創校歷史：** 創立於1966年，前身為明新工業專科學校。後於1997年改制為明新技術學院，並在2002年升格為科技大學。其發展歷程反映了台灣技職教育從專科到科技大學的轉型趨勢。

2.  **地理位置優勢：** 學校毗鄰新竹科學園區及湖口工業區，這使得明新科大在產學合作、學生實習與畢業就業方面具有得天獨厚的地理優勢，為在地及周邊產業輸送了大量專業人才。

3.  **教育理念：** 明新科大以「務實致用、學用合一」為教育目標，致力於培養具備專業技能、實作能力與人文素養的產業人才。課程設計強調理論與實務的結合，注重動手操作與解決問題的能力。

4.  **學術領域：** 學校設有多個學院，涵蓋了多元領域，主要包括：
    *   **工程學院：** 提供機械、電機、電子、土木等相關工程學系。
    *   **管理學院：** 涵蓋工業工程與管理、企業管理、資訊管理等。
    *   **設計學院：** 包含多媒體與遊戲發展、空間設計、時尚造型等。
    *   **服務事業學院：** 設有旅館事業管理、餐飲管理、幼兒保育等學系。

5.  **產學合作與就業：** 明新科大與產業界關係緊密，積極推動產學合作計畫，為學生提供豐富的實習機會和就業輔導。畢業生在就業市場上具有良好的競爭力，廣受企業歡迎。

6.  **特色發展：** 在新竹地區的科技產業背景下，明新科大特別加強在智慧科技、半導體相關技術、物聯網、大數據等領域的發展與人才培育。

總體而言，明新科技大學是一所深耕技職教育、強調實作訓練、與產業緊密接軌的綜合型科技大學，致力於為國家及社會培養具備即戰力的專業人才。


In [9]:
result2 = stateful_query("校長是誰？")
print(result2)

截至我上次知識更新（2023年中），明新科技大學的現任校長是 **劉國偉 博士**。

您可以隨時查閱明新科技大學的官方網站，以獲取最即時、最準確的資訊。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 15:42:44] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[]}
BODY:  {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"text","id":"594235075563094431","quoteToken":"otcxDih7eG0yHiVmwiVoL6EFrYKwqSGEx5NTgaJ23Aqrn0vm4_jVSWQvEeS_BieTWsaTEmGh7SX9XtRnscqyhaoA506fSVHl3NHzSmUAUXqMm76hAeWd4pZjBace_aItS9X2uEc7EUDyKq2hehp9Sg","markAsReadToken":"6ZjbR_ju8mofmq4Uv8LmJtOPnSXXCkbXFYZKF5oyz2iMou9XF_HgakTLxh4ocoxv4eRIgKTA0Wc_N-5OLn_Vm0DeK4QN-5zOPf1KGzB2QoGcNZaHkx_ifnbcJ7Nu86LFFdXs3w5608NACgDgPCvZbtpIJ0OuspoW3FxjzHERLfUl-IiH3eyI5rAxdIHZ6RIEDVkn1KlRLpGdis--1F_FrA","text":"AI 校長是誰"},"webhookEventId":"01KDNCERRWNK2S3PHTD9B5ZJQ4","deliveryContext":{"isRedelivery":false},"timestamp":1767023010079,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"d5d0b3649cd14243a60f7316d3efe5ca","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 15:43:33] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"text","id":"594235128142627334","quoteToken":"HvMkTGMLmckxnMoyqS19WmteyT00l67N0RKZmr3oEXamUfzL5U8NQm2HZQwxMrj4QJy-nfXN8j73Ymcvd25ITBXdfgzh4-5sVIWvLojy5IGPZ7vTn_hDMHSx2GRQV5CJLgp0L5ZlWK_Daz-t0LsQCQ","markAsReadToken":"-hMOPE_fH0M9_MZDI4_EIpqzuH8dkWSVzxes5F_DRlhFk3MN9sCks6mJ9f4c4gT_KT0p1iszsru5W7Ubj9Meu7Owdkb-xtZMDIfgF1PzxLuPKFn6my5Nwbg7RxPCDR1qgkFPaRX6Ea6GdU7t5iaEsU_ozeD1iergWFdGuLNuCb6BAKytzriyoQjqV_lcYxdms7HOtpQR0EqQDuCdqfV9mw","text":"AI 今天天氣是怎樣"},"webhookEventId":"01KDNCFQAWTX6RV417Z0Z2915H","deliveryContext":{"isRedelivery":false},"timestamp":1767023041371,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"d3fae10cf93d40deab444a0ed510d4b3","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 15:44:08] "POST / HTTP/1.1" 200 -
